### Disclaimer: portions of the below code were developed using ChatGPT

In [ ]:
#Run this if needed
#!pip install geopandas

In [12]:
import requests
import csv
import arcpy
from arcpy import env
import os
import numpy as np
import io
import json
from io import StringIO
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
import zipfile
from osgeo import gdal
import geopandas
import pyproj

### Elevation Data

In [ ]:
# To begin, we will acquire elevation data from the MN GeoCommons 
url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/elev_30m_digital_elevation_model/fgdb_elev_30m_digital_elevation_model.zip"

# We use the above url to send a GET request
response = requests.get(url, stream=True)

# Then we check if the request was successful (status code 200)
if response.status_code == 200:
    # Get the total file size in bytes
    total_size = int(response.headers.get('content-length', 0))
    # Initialize a tqdm progress bar with the total size (this is a big download so it helps to monitor progress)
    progress_bar = tqdm(total=total_size, unit='B', unit_scale=True)

    # We can specify the directory to save the file
    save_path = os.path.join("C:\\Users\\conno\\OneDrive\\Desktop\\GIS 5572", "fgdb_elev_30m_digital_elevation_model.zip")

    # And then actually do the content saving
    with open(save_path, "wb") as f:
        for data in response.iter_content(chunk_size=1024):
            # Write data to file
            f.write(data)
            # Update the progress bar with the size of the written data
            progress_bar.update(len(data))
    
    # We can then close the progress bar
    progress_bar.close()
    print("Download completed successfully.")
else:
    print("Failed to download:", response.status_code)

In [27]:
# In this next cell, we begin by specifying the path to the downloaded zip file we just downloaded
zip_file_path = r"C:\Users\conno\OneDrive\Desktop\GIS 5572\fgdb_elev_30m_digital_elevation_model.zip"

# Then we specify the directory where we want to extract the contents
extract_dir = r"C:\Users\conno\OneDrive\Desktop\GIS 5572"

# We open the zip file...
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # ... and extract all the contents to the specified directory
    zip_ref.extractall(extract_dir)

print("Extraction completed successfully.")

Extraction completed successfully.


In [ ]:
# For our QA/QC with our elevation data, we will check for the presence of null data.
raster_name = 'digital_elevation_model_30m'

# We first create a raster object
raster = arcpy.Raster(raster_name)

# Then we get the minimum and maximum values of the raster dataset
min_value = raster.minimum
max_value = raster.maximum

# Then we get the extent of the raster dataset
raster_extent = raster.extent

# We can then check is if the raster dataset contains null values
if min_value is None or max_value is None:
    print("The raster dataset contains null values.")
else:
    print("The raster dataset does not contain null values.")

In [ ]:
#We need to prepare the data to be stored in a PostGIS Database. First, we need to resample the raster to take up less space.
arcpy.management.Resample(
    in_raster="digital_elevation_model_30m",
    out_raster=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Lab 2\GIS 5572 Lab 2.gdb\digital_elevation_m_Resample",
    #We will be using 5 square km pixels
    cell_size="5000 5000",
    resampling_type="NEAREST"
)
print("Resample complete")

In [ ]:
#Then we need to convert the raster to points
# Input parameters
inRaster = "digital_elevation_m_Resample"
outPoint = "elevation_point"
field = "VALUE" #This denotes elevation

# Then we run the RasterToPoint arcpy function
arcpy.conversion.RasterToPoint(inRaster, outPoint, field)
print("Raster converted to points")

In [ ]:
# Before we save it, let's do another check to make sure the data is there

point_fc_name = 'elevation_point'

# We will count the number of features in the point feature class
num_features = arcpy.GetCount_management(point_fc_name)[0]

# Then we will check if the feature class does contain any features
if int(num_features) == 0:
    print("The point feature class is empty.")
else:
    print("The point feature class contains {} features.".format(num_features))

In [ ]:
# Now we can begin saving the dataset.
fc_name = 'elevation_point'
# To do this, I will convert the feature class into a dataframe
# I start off by creating an empty list to store the feature attributes
data = []

# Then we will iterate through the feature class and fetch attributes
fields = [field.name for field in arcpy.ListFields(fc_name)]
with arcpy.da.SearchCursor(fc_name, fields) as cursor:
    for row in cursor:
        data.append({fields[i]: row[i] for i in range(len(fields))})

# Then we will convert the list of dictionaries to a DataFrame
elevation_df = pd.DataFrame(data)

# To check that Display the DataFrame
elevation_df

In [59]:
#With things taken care of with the dataframe, we can use psychopg2 to save it to my PostGIS database
db_params = {
    'database': 'INSERT DATABASE',
    'user': 'INSERT USERNAME',
    'password': 'INSERT PASSWORD',
    'host': 'INSERT HOST ADDRESS',
    'port': 'INSERT PORT'
}

# First we create a connection to the PostgreSQL database
conn = psycopg2.connect(**db_params)

# Then we define the name of the PostGIS table that will be created
table_name = 'elevation_point_postgis'

# Then we define the SQLAlchemy engine
engine = create_engine('postgresql+psycopg2://' + db_params['user'] + ':' + db_params['password'] +
                       '@' + db_params['host'] + ':' + db_params['port'] + '/' + db_params['database'])

# We can then convert the elevation_df to PostGIS table
elevation_df.to_sql(table_name, engine, if_exists='replace', index=False, schema='public')

# Then we close the connection
conn.close()

### Landcover

In [ ]:
# Our next dataset is land classification data from MN GeoCommons
# We will use another API call in order to grab the needed zipfile
api_url="https://gisdata.mn.gov/api/3/action/package_show?id=biota-landcover-nlcd-mn-2019"
response = requests.get(api_url, verify=False)
# We can then convert the grabbed response into a JSON
landuse_json=response.json()

In [ ]:
# We then need to look for the TIF file, which we will do by looking through the JSON
resources = landuse_json['result']['resources']
for resource in resources:
    if resource['format'] == 'tif':
        #Once we find the shapefile, we can save the associated URL.
        zip_url = resource['url']
        #Next, we can save the zip_url to our directory as a zip file.
        zip_filename = os.path.basename(zip_url)

        # We can finally download the zip file and produce some code that can will generate a text output indicating the name of the file we have downloaded.
        response = requests.get(zip_url)
        if response.status_code == 200:
            with open(zip_filename, 'wb') as file:
                file.write(response.content)
            print(f"Downloaded {zip_filename}")
        else:
            print(f"Failed to download {zip_filename}. Status code: {response.status_code}")
        break 

In [ ]:
# Then, we of course need to unzip the file
zip_file_path = r"tif_biota_landcover_nlcd_mn_2019.zip"

#Then we define the folder where we want to extract the contents
extracted_folder = r"C:\Users\conno\OneDrive\Desktop\GIS 5572"

#Then we can open the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)

print(f"All files have been extracted")

In [ ]:
# Set local variables
arcpy.management.Resample(
    in_raster="NLCD_2019_Land_Cover.tif",
    out_raster=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Lab 2\GIS 5572 Lab 2.gdb\landcover_Resample",
    cell_size="5000 5000",
    resampling_type="NEAREST"
)
print("Raster has been resampled")

In [ ]:
# Set local variables
inRaster = "landcover_Resample"
outPoint = "landcover_point"
field = "NLCD_Land"

# Run RasterToPoint
arcpy.conversion.RasterToPoint(inRaster, outPoint, field)

print("Raster to point conversion complete")

In [ ]:
# For QA/QC purposes, let's check and see how many points have actual classes
feature_class = r'C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Lab 2\GIS 5572 Lab 2.gdb\landcover_point'

# We will first create an empty list to store the point IDs of unclassified points
unclassified_point_ids = []

# Then we will count the total number of points
total_points = int(arcpy.GetCount_management(feature_class)[0])

# We can then initialize a counter for unclassified points
unclassified_count = 0

# Next, we need to loop through the points to gather Point ID and Land Class info
with arcpy.da.SearchCursor(feature_class, ['PointID', 'NLCD_Land']) as cursor:
    for row in cursor:
        point_id, nlcd_land = row
        if nlcd_land == 'Unclassified':
            unclassified_point_ids.append(point_id)
            unclassified_count += 1

# Then we calculate the percentage of unclassified points and print it out
percentage_unclassified = (unclassified_count / total_points) * 100
print(f"Percentage of points classified as 'unclassified': {percentage_unclassified:.2f}%")

# Then we save the point IDs of unclassified points to a separate dataframe
unclassified_points_df = pd.DataFrame({'PointID': unclassified_point_ids})

# We can also save the dataframe with point IDs of unclassified points to a CSV file
unclassified_points_df.to_csv('unclassified_points.csv', index=False)

# Finally, let's check the contents of this dataframe
print(unclassified_points_df)

In [69]:
# With the data ready, let's save it to a PostGIS table
# Like with elevation, we first define the feature class name
fc_name = 'landcover_point'

# We then create an empty list to store the feature attributes
data = []

# Then we iterate through the feature class and fetch attributes
fields = [field.name for field in arcpy.ListFields(fc_name)]
with arcpy.da.SearchCursor(fc_name, fields) as cursor:
    for row in cursor:
        data.append({fields[i]: row[i] for i in range(len(fields))})

# Then we convert the list of dictionaries to a DataFrame
landcover_df = pd.DataFrame(data)

# Display the DataFrame
landcover_df

#Outline the database parameters
db_params = {
    'database': 'INSERT DATABASE',
    'user': 'INSERT USERNAME',
    'password': 'INSERT PASSWORD',
    'host': 'INSERT HOST ADDRESS',
    'port': 'INSERT PORT'
}

# Create a connection to the PostgreSQL database
conn = psycopg2.connect(**db_params)

# Define the name of the PostGIS table
table_name = 'landcover_point_postgis'

# Define the SQLAlchemy engine
engine = create_engine('postgresql+psycopg2://' + db_params['user'] + ':' + db_params['password'] +
                       '@' + db_params['host'] + ':' + db_params['port'] + '/' + db_params['database'])

# Convert DataFrame to PostGIS table
landcover_df.to_sql(table_name, engine, if_exists='replace', index=False, schema='public')

# Close the connection
conn.close()

### Potential Evapotranspiration

In [ ]:
# This data is dependent on date
# First we will prompt the user for start and end dates of the data to be called
start_date = input("Enter the start date (YYYY-MM-DD): ")
end_date = input("Enter the end date (YYYY-MM-DD): ")

# Then this code constructs the URL using the user-input dates
url = f"https://ndawn.ndsu.nodak.edu/table.csv?station=78&station=174&station=118&station=87&station=124&station=184&station=2&station=93&station=183&station=156&station=70&station=173&station=185&station=187&station=119&station=4&station=82&station=120&station=71&station=103&station=116&station=114&station=3&station=115&station=121&station=61&station=181&station=60&station=122&station=5&station=91&station=182&station=117&station=6&station=92&station=123&station=95&station=148&variable=ddtpetp&year=2024&ttype=daily&quick_pick=&begin_date={start_date}&end_date={end_date}"

# We get the response
response = requests.get(url)

# And we can check if the request was successful
if response.status_code == 200:
    print("Data call works!")
else:
    print("Data call failed. Status code:", response.status_code)

In [23]:
# This data looks through and filters out the needed info from the URL response
data = response.text
lines = data.strip().split('\n')[3:]
result = '\n'.join(lines)
csv_file = StringIO(result)
dataframe = pd.read_csv(csv_file)
# In the end, it provides a dataframe of the acquired station data
dataframe = dataframe.iloc[1:]

In [ ]:
# Let's clean out any with potential NaN values.
# First, let's establish a new separate dataframe
nan_values_df = dataframe[dataframe[['Latitude', 'Longitude', 'Penman PET']].isna().any(axis=1)]


# We will ensure that the gathered latitude, longtiude values in this dataframe are numeric
dataframe['Latitude'] = pd.to_numeric(dataframe['Latitude'], errors='coerce')
dataframe['Longitude'] = pd.to_numeric(dataframe['Longitude'], errors='coerce')
dataframe['Penman PET'] = pd.to_numeric(dataframe['Penman PET'], errors='coerce')

# We will then go through and clean the data of any NaN values
df = dataframe.dropna(subset=['Penman PET'])

# Then we can print the Data Frame entries without NaN values
print("DataFrame without NaN values:")
print(df)

# Then we can print the separate Data Frame entries containing rows with NaN values
print("\nDataFrame with rows containing NaN values:")
print(nan_values_df)

In [ ]:
# We will condense the viable data frame by selecting specific columns to keep
columns_to_keep = ['Station Name', 'Latitude', 'Longitude', 'Penman PET']
df_modified = df[columns_to_keep]

# Show modified DataFrame
print(df_modified)

In [ ]:
# Let us also check each reporting station is within Minnesota
minnesota_bbox = (-97.239209, 43.499356, -89.491739, 49.384358)

# We use the latitude and longtitude columns of the data frame to filter stations outside the bounding box
stations_outside_bbox = df_modified[
    (df_modified['Latitude'] < minnesota_bbox[1]) |
    (df_modified['Latitude'] > minnesota_bbox[3]) |
    (df_modified['Longitude'] < minnesota_bbox[0]) |
    (df_modified['Longitude'] > minnesota_bbox[2])
]

# We can print the names of stations that fall outside the bounding box here
if not stations_outside_bbox.empty:
    print("Stations outside Minnesota bounding box:")
    print(stations_outside_bbox['Station Name'].tolist())
else:
    print("All stations are within Minnesota bounding box.")

In [ ]:
# To save this data to PostGIS database, I am converting the points to a vertices dictionary
vertices = []
for index, row in df_modified.iterrows():
    # I can then extract information for each station
    name = row['Station Name']
    latitude = row['Latitude']
    longitude = row['Longitude']
    PET = row['Penman PET']
    # And then create a entry for each station with its latitude, longitude, name, and PET value
    vertex = (longitude, latitude, name, PET)
    # I can then append the vertex entry to the vertices list
    vertices.append(vertex)

print(vertices)

In [ ]:
# Once I have the vertices list saved as a feature class, I can visualize on ArcGIS Pro if needed
workspace = arcpy.env.workspace
spatial_reference = arcpy.SpatialReference(4326)  # WGS 1984 (EPSG code: 4326) or specify your desired coordinate system

fc_name = "PET_Points"

# We create a feature class to store the points
arcpy.management.CreateFeatureclass(workspace, fc_name, "POINT", spatial_reference=spatial_reference)

# Then we add a field to store the name and PET values
arcpy.management.AddField(fc_name, "Name", "TEXT")
arcpy.management.AddField(fc_name, "PET", "FLOAT")

# Then we create an array to store point geometries
array = arcpy.Array()

# Iterate over the vertices
for latitude, longitude, name, pet in vertices:
    # Create a point geometry
    point = arcpy.Point(longitude, latitude)
    array.append(point)
    
    # Insert the point feature into the feature class
    with arcpy.da.InsertCursor(fc_name, ["SHAPE@", "Name", "PET"]) as cursor:
        cursor.insertRow([point, name, pet])

# Then we create a point feature from the array and insert it into the feature class
with arcpy.da.InsertCursor(fc_name, ["SHAPE@", "Name", "PET"]) as cursor:
    for point, name, pet in zip(array, [vertex[2] for vertex in vertices], [vertex[3] for vertex in vertices]):
        cursor.insertRow([point, name, pet])

# Finally, we add the feature class to the current ArcGIS Pro map
aprx = arcpy.mp.ArcGISProject("CURRENT")
map = aprx.listMaps()[0]
map.addDataFromPath(fc_name)

print("Point features created and added to the map.")

In [83]:
# Then, much like the last two datasets, I can save the dataframe to a PostGIS table
db_params = {
    'database': 'INSERT DATABASE',
    'user': 'INSERT USERNAME',
    'password': 'INSERT PASSWORD',
    'host': 'INSERT HOST ADDRESS',
    'port': 'INSERT PORT'
}


conn = psycopg2.connect(**db_params)
table_name = 'pet_point_postgis'
engine = create_engine('postgresql+psycopg2://' + db_params['user'] + ':' + db_params['password'] +
                       '@' + db_params['host'] + ':' + db_params['port'] + '/' + db_params['database'])
df_modified.to_sql(table_name, engine, if_exists='replace', index=False, schema='public')
conn.close()

### Temperature

In [ ]:
# Our final dataset is courtesy of Mesonet.
# The following CSV produces a CSV of all stations in the RWIS network in Minnesota
url = "https://mesonet.agron.iastate.edu/sites/networks.php?network=MN_RWIS&format=csv&nohtml=on"

# Read the CSV file into a DataFrame
station_df = pd.read_csv(url)

# Define boundaries for Minnesota
min_x, min_y, max_x, max_y = -97.5, 43.0, -89.0, 49.5

# We can check if latitude and longitude fall within Minnesota boundaries
within_minnesota = (station_df['lon'] >= min_x) & (station_df['lon'] <= max_x) & \
                   (station_df['lat'] >= min_y) & (station_df['lat'] <= max_y)

# If needed, we can get names of stations outside Minnesota boundaries
outside_minnesota = station_df[~within_minnesota]['stid']

# Finally, we print "yes" if all stations fall within Minnesota, otherwise print "no" and list the names of stations outside the boundaries
if within_minnesota.all():
    print("Yes, everything is within bounds")
else:
    print("No, there are some stations out of bounds")
    print("Stations outside Minnesota boundaries:")
    for station_name in outside_minnesota:
        print(station_name)

In [ ]:
# # This code looks through the station_df and inputs the station IDs into the below URL with a user inputted data
def fetch_data_for_date(station_id, date, lat, lon):
    url = f"https://mesonet.agron.iastate.edu/api/1/obhistory.json?station={station_id}&network=MN_RWIS&date={date}&full=1"
    response = requests.get(url)
    data = response.json()
    
    # Check if data is available for the station
    if 'data' in data:
        # Convert data to DataFrame
        station_data_df = pd.DataFrame(data['data'])
        
        # Add station ID, latitude, and longitude columns
        station_data_df['station_id'] = station_id
        station_data_df['lat'] = lat
        station_data_df['lon'] = lon
        
        return station_data_df
    else:
        return pd.DataFrame()

# We can create an empty list to store data frames
data_frames = []

# This will prompt the user to input the date
date = input("Enter the date (YYYY-MM-DD): ")

# Then this code will go through each station ID in the first dataset
for index, row in station_df.iterrows():
    station_id = row['stid']
    lat = row['lat']
    lon = row['lon']
    
    # Fetch data for the specified date
    station_data_for_date = fetch_data_for_date(station_id, date, lat, lon)
    
    # And then append station data for the specified date to the list of data frames
    data_frames.append(station_data_for_date)

# Concatenate all data frames in the list into one dataframe
combined_df = pd.concat(data_frames, ignore_index=True)

# Display the combined DataFrame
combined_df

In [ ]:
# To summarize the data, this code averages out the air temperature value for each station
averaged_df = combined_df.groupby('station_id').agg({'tmpf': 'mean', 'lat': 'first', 'lon': 'first'}).reset_index()

print(averaged_df)

In [ ]:
# The next code checks for outliers based on calculated z-score
# It doesn't remove the data--simply creates a brief list of any outliers and their station names
averaged_df['tmpf_zscore'] = (averaged_df['tmpf'] - averaged_df['tmpf'].mean()) / averaged_df['tmpf'].std()

# Define threshold for outlier detection (e.g., z-score > 3 or < -3)
outlier_threshold = 3

# Identify outliers based on z-score
outliers = averaged_df[np.abs(averaged_df['tmpf_zscore']) > outlier_threshold]

# Check if the number of outliers exceeds 10% of the total number of stations
if len(outliers) > 0.1 * len(averaged_df):
    print("Number of outliers exceeds 10% of total stations. Script terminated.")
    # Optionally, you might want to add further actions if the condition is met, like logging or terminating the script.
    exit()  # This terminates the script execution

print("Outliers:")
print(outliers)

In [ ]:
temp_vertices = []
for index, row in averaged_df.iterrows():
    # Extracting information for each station
    name = row['station_id']
    latitude = row['lat']
    longitude = row['lon']
    air_temp = row['tmpf']

    # Creating a tuple for each station with its latitude, longitude, and name
    vertex = (longitude, latitude, name, air_temp)
    # Appending the vertex tuple to the vertices list
    temp_vertices.append(vertex)

print(temp_vertices)

In [ ]:
import arcpy

# Define the workspace and spatial reference
workspace = arcpy.env.workspace  # Use your desired workspace
spatial_reference = arcpy.SpatialReference(4326)  # WGS 1984 (EPSG code: 4326) or specify your desired coordinate system

# Define the name of the feature class
fc_name_temp = "Temperature_Points"

# Create a feature class to store the temperature points
arcpy.management.CreateFeatureclass(workspace, fc_name_temp, "POINT", spatial_reference=spatial_reference)



In [ ]:

# Add fields to store attributes
arcpy.management.AddField(fc_name_temp, "Station_ID", "TEXT")
arcpy.management.AddField(fc_name_temp, "Air_Temperature", "FLOAT")
arcpy.management.AddField(fc_name_temp, "Time", "TEXT")


# Create an array to store point geometries
array_temp = arcpy.Array()

for lon, lat, name, air_temp in temp_vertices:
    # Create a point geometry
    point_temp = arcpy.Point(lon, lat)
    array_temp.append(point_temp)
    
    # Insert the point feature into the feature class
    with arcpy.da.InsertCursor(fc_name_temp, ["SHAPE@", "Station_ID", "Air_Temperature"]) as cursor_temp:
        cursor_temp.insertRow([point_temp, name, air_temp])

# Add the feature class to the current ArcGIS Pro map
aprx = arcpy.mp.ArcGISProject("CURRENT")
map = aprx.listMaps()[0]
map.addDataFromPath(fc_name_temp)

print("Temperature point features created and added to the map.")

In [ ]:
#Finally we can save the temperature data to a PostGIS database
db_params = {
    'database': 'INSERT DATABASE',
    'user': 'INSERT USERNAME',
    'password': 'INSERT PASSWORD',
    'host': 'INSERT HOST ADDRESS',
    'port': 'INSERT PORT'
}


conn = psycopg2.connect(**db_params)
table_name = 'temp_point_postgis'
engine = create_engine('postgresql+psycopg2://' + db_params['user'] + ':' + db_params['password'] +
                       '@' + db_params['host'] + ':' + db_params['port'] + '/' + db_params['database'])
averaged_df.to_sql(table_name, engine, if_exists='replace', index=False, schema='public')
conn.close()